##  QUESTO E' IL FILE DI RIDUZIONE VERO DI FEDERICO PIGNALBERI
#### I coeeficienti nelle funzioni di seguito sono ottenuti da una procedura di calibrazione, necessaria perche' le curve che si ottengono non sono lineari

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.stats import linregress
import matplotlib.pyplot as plt 
import sys
sys.path.insert(0,'/home/federico/PythonModules/rawPy-master/RawPy')
import rawPy as rp

# The arrays in these functions must be moved in the machines collections, under the
# "Vertical Piston" and "Horizontal Piston" fields. They are obtained by calibration procedures
# The functions theirself should be methods in machineDao class to be used to get the 
# centralized measures retrieved in an experiment executed with that specific machine: all the experiment
# created after that calibration procedure and before the next one must use the corresponding one to
# convert measure in volt to the right units
def horizontal_calibration(volt):
    '''
    input:
        Voltage (must be offsetted)
    Return: 
        Force (kN)
    '''
    x=np.array(volt)
    coefficients = np.array([-4.63355231e-02, -2.57055418e+00,  2.63055688e+01, -9.61932787e+01,
            1.64685122e+02, -1.33648859e+02,  4.66773182e+01,  1.63975941e+02,
            9.32438525e-02])
    hor_calibration = np.poly1d(coefficients)
    force = hor_calibration(x)   
    return force

def vertical_calibration(volt):
    '''
    input:
        Voltage (must be offsetted)
    Return: 
        Force (kN)
    '''
    x=np.array(volt)
    coefficients = np.array([ -0.5043737 ,   4.27584024, -11.70546934,   5.45745069,
            29.43390033, -60.90428874,  60.98729795, 124.19783947,
            -0.47000267])
    vert_calibration = np.poly1d(coefficients)
    force = vert_calibration( x)
    return force
######################## ELASTIC CORRECTION FUNCTIONS
def vertical_stiffness (force):
    '''
    input:
        Force (in kN)
    Return: 
        Stiffness array
    '''
    coefficients = np.array([ 3.35241499e-30, -9.37367134e-27,  1.19440060e-23, -9.17845027e-21,
            4.74088379e-18, -1.73500082e-15,  4.61575497e-13, -9.00528796e-11,
            1.28295415e-08, -1.31327065e-06,  9.38895324e-05, -4.50176164e-03,
            1.38008389e-01, -2.63525139e+00,  3.57750394e+01,  1.71503762e+01])
    calibration = np.poly1d(coefficients)
    stiffness = calibration(force)
    return stiffness

def horizontal_stiffness (force):
    '''
    input:
        Force (in kN)
    Return: 
        Stiffness array
    '''
    coefficients = np.array([ 2.43021220e-31, -7.73507440e-28,  1.10791696e-24, -9.43050473e-22,
            5.30556343e-19, -2.07533887e-16,  5.77817817e-14, -1.15148744e-11,
            1.62528123e-09, -1.57483543e-07,  9.75756659e-06, -3.16390679e-04,
            1.96801181e-04,  2.69515293e-01,  5.53939566e+00,  4.21560673e+01])
    calibration = np.poly1d(coefficients)
    stiffness = calibration(force)
    return stiffness

def elastic_correction(stress, disp, k):
    """
    Apply elastic correction to displacement data.

    Parameters:
    stress (np.ndarray): The stress data array.
    disp (np.ndarray): The displacement data array.
    k (float or np.ndarray): The stiffness value (scalar or array).

    Returns:
    np.ndarray: Elastic corrected displacement data.
    """
    # Convert scalar k to an array if it is not already an array
    if np.isscalar(k):
        k = np.full_like(stress[:-1], k)
    else:
        k = k[:-1]

    # Increments in elastic distortion
    dload = (stress[1:] - stress[:-1]) / k
    # Increments in total displacement
    ddisp = disp[1:] - disp[:-1]
    # Subtract elastic distortion from total displacement
    ec_disp = np.hstack([0, np.cumsum(ddisp - dload)])
    return ec_disp


############################################
experiment_id = 's0144sa03min22'

# To get the information from database
# Here must be used "get_centralized_measurements",
# instead of the rp.load_tdms() method
path = os.getcwd()
df = rp.load_tdms('%s/%s.tdms'%(path,experiment_id))
Vertical_Load_recorded = df.Vertical_Load
Horizontal_Load_recorded = df.Horizontal_load
Vertical_Displacement = df.Vertical_Displacement
Horizontal_Displacement = df.Horizontal_Displacement
time_s  = df.Time
records_na = df.Rec_n     # this is obtained from the sequential number of Time, so superflous

# these values are in the "properties" of "vertical Load" and "Horizontal Load" 
# in the "ADC" group properties. They are reported as "slope" and "intercept"
# They are not real calibration values, as for the displacement ones
# The scaling value are for visualization purpose, so the recoerded measures 
# must be reported to Volt values before true calibration scaling. 
# non-linear calibration scaling is performed using the coefficient saved in the database as calibration
# These coefficients are just the one Federico Pignalberi put in the preceding functions and come from 
# last calibration of Brava2 pistons 
vertical_visualization_scaling = 100 
vertical_intercept = -37.0619  
horizontal_visualization_scaling = 100 
horizontal_intercept = 132.026

vertical_load_voltage = (Vertical_Load_recorded-vertical_intercept)/vertical_visualization_scaling
horizontal_load_voltage = (Horizontal_Load_recorded-horizontal_intercept)/horizontal_visualization_scaling

vertical_load_kN = vertical_calibration(vertical_load_voltage)
horizontal_load_kN = horizontal_calibration(horizontal_load_voltage)

# This information can be retrieved from the lateral block sizes: gouge area is just x*y dimensions 
gouge_area = 0.0025 # m^2. It can be retrieved from the dimensions of the blocks 
conversion_parameter = 1e3 * gouge_area # from kN to MPa, so from force to stress


#### Vertical stress ###
#### This is the point at which the ram contacted the blocks and load shear loading began
#### So far it is chosen looking at the plot, but it must be automatize too
beg_row_v = 2808
##################################

# ZERO DATA
# add in force from the central block due to gravity 
# v_load = v_load + 0.044 #[kN]
# remove noise before load is applied
vertical_load_kN = vertical_load_kN - vertical_load_kN[beg_row_v]
vertical_load_kN[:beg_row_v]= vertical_load_kN[:beg_row_v]*0               
# calculate stress for DDS 5x5
shear_stress_MPa = vertical_load_kN/(2*conversion_parameter)      # why that "2" Lo sforzo di taglio va suddiviso sui due gouges, non cosi quello normale!!!

# Note:  
# 1. These values are calculated for a 5x5[cm] shear surface  
# 2. Stiffness is non linear at low applied loads  
# TO DO: implement a function that corrects for non-linear stiffness: this has been done by Pignalberi
# Here the vertical force is zeroed 
vertical_force = np.where(vertical_load_kN > 0, vertical_load_kN, 0)
#### vertical disp ####
load_point_disp = Vertical_Displacement - Vertical_Displacement[beg_row_v] # [mum]
load_point_disp[:beg_row_v] = load_point_disp[:beg_row_v]*0
load_point_disp_mm = load_point_disp/1000       # [mm]

# Elastic correction for the stretch of the vertical frame # 
vertical_k_constant = 166.801 #[MPa/mm]         # calibration from loading and unloading cycles on steel: linear approx
vertical_k = vertical_stiffness(vertical_force) # Pignalberi entire curve calibration

# "Elastic corrected" displacement
ec_disp_mm = elastic_correction(vertical_force,load_point_disp_mm,vertical_k) 
ec_disp_mm_old = elastic_correction(shear_stress_MPa,load_point_disp_mm,vertical_k_constant)

#############  normal load ############ 
#### Normal load is applied at record
beg_row_h = 234

######################################
# zero the data
horizontal_load_kN = horizontal_load_kN - horizontal_load_kN[beg_row_h]
# remove noise before load is applied 
horizontal_load_kN[:beg_row_h] = horizontal_load_kN[:beg_row_h]*0
# calculate stress for DDS 5x5
normal_stress_MPa =  horizontal_load_kN/(conversion_parameter)
#add a small number to normal stress 
#so that we aren't dividing by 0 anywhere to calculate mu 
epsilon = 1e-7
normal_stress_MPa = normal_stress_MPa + epsilon

# Note:  
# 1. These values are calculated for a 5x5[cm] shear surface  
# 2. Stiffness is non linear at low applied loads  
# TO DO: implement a function that corrects for non-linear stiffness. Pignalberi has done it
horizontal_force = np.where(vertical_load_kN > 0, vertical_load_kN, 0)

# Elastic correction for the stretch of the horizontal frame #
horizontal_k_constant = 125.363 #[MPa/mm]              # calibration from loading and unloading cycles on steel: linear approx
horizontal_k = horizontal_stiffness(horizontal_force)  # Pignalberi entire curve calibration

# Carolina, correzione iniziale serve nel caso carico normale variabile e se necessario valore assoluto layer thickNess (tipo, fase iniziale)
horizontal_displacement_corrected_mm = elastic_correction(horizontal_force,-Horizontal_Displacement/1000,horizontal_k)   # why "-"? So that compaction is negative
horizontal_displacement_corrected_mm_old = elastic_correction(normal_stress_MPa,-Horizontal_Displacement/1000,horizontal_k_constant)

######################################################################
########## 5. Calculate layer thickNess: MUST STANDARDIZE!!! ##########################
# ##################################################################### 
# insert rec number at which layer thickness was measured
rec_lt = 234
# insert calculated value at that point
val_lt = 3    # [mm]
###########################################################
# zero data
layer_thickness_corrected_mm = horizontal_displacement_corrected_mm - horizontal_displacement_corrected_mm[rec_lt]
layer_thickness_corrected_mm_half = layer_thickness_corrected_mm/2
layer_thickness_corrected_mm1 = layer_thickness_corrected_mm_half + val_lt

layer_thickness_corrected_mm_old = horizontal_displacement_corrected_mm_old - horizontal_displacement_corrected_mm_old[rec_lt]
layer_thickness_corrected_mm_old_half = layer_thickness_corrected_mm_old/2
layer_thickness_corrected_mm_old1 = layer_thickness_corrected_mm_old_half + val_lt

# default unit is [mm]
rgt_lt_mm = rp.rgt(load_point_disp_mm,layer_thickness_corrected_mm1)     # remove geometrical thinning
# # 6. Calculate friction 
friction_na = shear_stress_MPa/normal_stress_MPa


# # 7. Calculate shear strain. Marco dice indifferente lt o quella corretta per geometrical thinning
shear_strain_na = rp.shear_strain(ec_disp_mm,layer_thickness_corrected_mm1)


ModuleNotFoundError: No module named 'rawPy'

In [ ]:
# EXPORT DATA
exp_name ='s144'
data_out = [shear_stress_MPa,
            load_point_disp_mm,
            ec_disp_mm, 
            #ob_disp,
            #ed_left_µm,
            #ed_right_µm,
            #ed_vert_µm,
            normal_stress_MPa,
            layer_thickness_corrected_mm1, 
            rgt_lt_mm,
            friction_na,
            shear_strain_na, 
            time_s,
            records_na
          ]
            
rp.save_data(exp_name,data_out,callingLocals=locals())